# ADS Capstone Project: Airbnb Pricing Prediction
#### By Renetta Nelson, Michael Nguyen and Jacqueline Urenda

# Modeling

In [19]:
#importing libraries

import pandas as pd
import numpy as np
import random
import xgboost
from xgboost import XGBRegressor


from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from tabulate import tabulate

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [2]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test


## Decision Tree Modeling

#### Setting baseline decision tree model

In [3]:
#fitting baseline regressor

DCTR = DecisionTreeRegressor()
DCTR.fit(X_train,y_train)


DecisionTreeRegressor()

#### Evaluating performance of baseline model

In [4]:
y_pred = DCTR.predict(X_test)

In [5]:
y_pred_train = DCTR.predict(X_train)

In [6]:
MSE = mean_squared_error(y_test, y_pred)
RMSE = mean_squared_error(y_test, y_pred, squared = False)
R2 = r2_score(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)


n = len(y_test)
k = len(X_test.columns)
Adjusted_R2 = 1-(((1-R2)*(n-1))/(n-k-1))

In [7]:
DCTR_Results = [['MSE', 'RMSE', 'R2', 'Adjusted_R2', 'MAPE'], 
         [MSE, RMSE, R2, Adjusted_R2, MAPE]]

print(tabulate(DCTR_Results))

-----------------  ------------------  ------------------  ------------------  ------------------
MSE                RMSE                R2                  Adjusted_R2         MAPE
66934.93773981836  258.71787286505423  0.9620149641003602  0.9620139857349796  0.1828629788219595
-----------------  ------------------  ------------------  ------------------  ------------------


## XGBoost Rgression Model

###

In [10]:
import xgboost
from xgboost import XGBRegressor

In [13]:
XGBR = XGBRegressor()

XGBR.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [14]:
y_pred_XGB = XGBR.predict(X_test)

In [17]:
XGB_MSE = mean_squared_error(y_test, y_pred_XGB)
XGB_RMSE = mean_squared_error(y_test, y_pred_XGB, squared = False)
XGB_R2 = r2_score(y_test, y_pred_XGB)
XGB_MAPE = mean_absolute_percentage_error(y_test, y_pred_XGB)


n = len(y_test)
k = len(X_test.columns)
XGB_Adjusted_R2 = 1-(((1-R2)*(n-1))/(n-k-1))

In [18]:
XGB_Results = [['MSE', 'RMSE', 'R2', 'Adjusted_R2', 'MAPE'], 
         [XGB_MSE, XGB_RMSE,XGB_R2, XGB_Adjusted_R2, XGB_MAPE]]

print(tabulate(XGB_Results))

----------------  -----------------  ------------------  ------------------  ------------------
MSE               RMSE               R2                  Adjusted_R2         MAPE
72983.4207401773  270.1544386830935  0.9585824989085637  0.9620139857349796  0.3090160273277967
----------------  -----------------  ------------------  ------------------  ------------------


## Random Forrest Model

## Note: to include Random Forest Model Code Soon